In [118]:
import numpy as np
from cvxopt import matrix,solvers
import math 
import time

This file is specifically for travelling salesman, and here there are two inputs given, one is "n" , which is no. of cities or countries or nodes, while second is "use" which is 0 by default, and use Branch and Bound, while if it is not equal to 0,
 it will use Branch and Cut, ensure that n=3 to 8, for n>8 ,the program may crash.
 It will automatically generate symmetric matrix(so that distance between two nodes is same), and diagonal elements will
 be zero (so that distance with itself is zero)

In [119]:
start_time=time.time()
use=0
n=5
c=np.random.randint(1,100,size=(n,n))
c=np.abs((c-c.T)/2)*2
print(c)
c=np.reshape(c,(n**2))
print("\n",c)
solution=[]
iterations=[]

[[ 0. 15. 22.  9. 51.]
 [15.  0.  8. 41.  5.]
 [22.  8.  0.  1. 24.]
 [ 9. 41.  1.  0. 57.]
 [51.  5. 24. 57.  0.]]

 [ 0. 15. 22.  9. 51. 15.  0.  8. 41.  5. 22.  8.  0.  1. 24.  9. 41.  1.
  0. 57. 51.  5. 24. 57.  0.]


The following blocks are for preprocessing which will create A,B and C matrix by creating n^2+n variables and n^2+4n 
constraints and then further blocks are for Branch and Bound and Branch and Cut Algorithm, at bottom we will get results

In [120]:
def preprocessing_TSP(c):
    
    n=int(len(c)**0.5)
    constraint=np.zeros(n**2+n-1)
    constraint[0]=1
    A=constraint
    b=np.array([0])
    
    for i in range(n):
        constraint=np.zeros(n**2+n-1)
        constraint[n*i:n*i+n]=1
        A=np.vstack([A,constraint])
        b=np.append(b,1)
        constraint=np.zeros(n**2+n-1)
        constraint[n*i:n*i+n]=-1
        A=np.vstack([A,constraint])
        b=np.append(b,-1)
        
        constraint=np.zeros(n**2+n-1)
        for j in range(n):
            constraint[n*j+i]=1
        A=np.vstack([A,constraint])
        b=np.append(b,1)
        
        constraint=np.zeros(n**2+n-1)
        for j in range(n):
            constraint[n*j+i]=-1
        A=np.vstack([A,constraint])
        b=np.append(b,-1)
            
    for i in range(1,n):
        for j in range(1,n):
            if(i!=j):
                pass
                constraint=np.zeros(n**2+n-1)
                constraint[n*i+j]=n-1
                constraint[n**2-1+i]=1
                constraint[n**2-1+j]=-1
                A=np.vstack([A,constraint])
                b=np.append(b,n-2)
            else :
                constraint=np.zeros(n**2+n-1)
                constraint[(n+1)*i]=1
                A=np.vstack([A,constraint])
                b=np.append(b,0)
        constraint=np.zeros(n**2+n-1)
        constraint[n**2-1+i]=-1
        A=np.vstack([A,constraint])
        b=np.append(b,-1)
        constraint=np.zeros(n**2+n-1)
        constraint[n**2-1+i]=1
        A=np.vstack([A,constraint])
        b=np.append(b,n-1)

    c=np.hstack([c,np.zeros(n-1)])
    A=A.astype(np.float64)
    b=b.astype(np.float64)
    c=c.astype(np.float64)
    return c,b,A

In [121]:
def preprocessing_CP(c,a,b):
    Nc=np.shape(a)[0] #  no of constraints
    c=np.hstack((c,(np.zeros(Nc+1))))
    I=np.eye(Nc)
    tableau=np.append(a,I,axis=1)
    tableau=np.vstack([tableau.T,b])
    tableau=tableau.T
    tableau=np.vstack([tableau,c])
    return tableau

In [122]:
def preprocessing_BAB(c,a,b) :
    Nc=np.shape(a)[0]
    Nv=np.shape(a)[1]
    b=np.hstack((b,(np.zeros(Nv))))
    I=np.eye(Nv)
    a=np.append(a,-I,axis=0)
    c=matrix(c)
    b=matrix(b)
    a=matrix(a)
    return c,a,b
    

In [123]:
def simplex(a,A):
    fv=[]
    infeasible=0
    Nc=np.shape(A)[0]
    Nv=np.shape(A)[1]
    for i in range(Nc):
        fv.append(Nv+i)
    for k in range(1001):
        if(np.all(a[-1,0:-1]>=0) and np.all(a[0:-1,-1]>=0)):
            return a,fv,infeasible
        else :
            column=np.argmin(a[-1,0:-1]) 
            if a[-1][column]<0:
                b=(a[0:-1,column])
                b=np.where(b==0,0.0000000001,b)
                ratio=((a[0:-1,-1])/b)
                ratio=np.where(ratio<0.0001,100000,ratio)
                for pivot in range(len(ratio)):
                    if ratio[pivot]==0 and b[pivot]<=0 :
                        ratio[pivot]=10000
                row=np.argmin(ratio)
                a[row,:]=a[row,:]/a[row][column]
                fv[row]=column
                for i in range(Nc+1):
                    if(i!=row):
                        a[i,:]=a[i,:]-a[i][column]*a[row,:]
                continue
            else :
                for i in range(Nc):  
                    if(a[i][-1]<-0.000001):
                        column=np.argmin(a[i,:-1])
                        if a[i][column]<0:
                            row=i
                            b=(a[0:-1,column])
                            b=np.where(b==0,0.0000000001,b)
                            ratio=((a[0:-1,-1])/b)
                            ratio=np.where(ratio<0.01,10000,ratio)
                            for pivot in range(len(ratio)):
                                if ratio[pivot]>=0 and ratio[pivot]<row and b[pivot]>0 :
                                    row=pivot
                            a[row,:]=a[row,:]/a[row][column]
                            fv[row]=column
                            for j in range(Nc+1):
                                if(j!=row):
                                    a[j,:]=a[j,:]-a[j][column]*a[row,:]
                            i=0
                        else :
                            infeasible=1
                            return a,fv,infeasible
    return a,fv,infeasible
            

In [124]:
def onecut(A,B,C):
    Nv=np.shape(A)[1] # no of variables
    flag=0
    Nc=np.shape(A)[0]
    D=np.zeros([Nv,Nv+1])
    for i in range(Nv):
        D[i,i]=-1
    E=np.vstack((A.T,B))
    E=E.T
    D=np.vstack((D,E,np.zeros(Nv+1)))
    D[-1,-1] = -1  # Size of D = Nc+1
    tableau=preprocessing_CP(C,A,B)
    sol_tab,solutions,infeasible=simplex(tableau,A)
    #sol_tab=np.round(sol_tab,5)
    z=sol_tab-np.floor(sol_tab)
    x=np.zeros(Nv)
    for j in range(Nv) :
        for i in range(len(solutions)):
            if solutions[i]==j:
                x[j]=sol_tab[i,-1]
                
    new_const=np.zeros(Nv+1)
    
    if(infeasible==1) or np.any(sol_tab[-1,0:-1]<0) or np.any(sol_tab[0:-1,-1]<0):
        flag=100
        return A,B,x,sol_tab[-1][-1],flag,new_const
        
    if np.all(z[0:-1,-1]<=0.001) :
        flag=1
    
    else :
        k=0
        for i in range(np.shape(z)[0]-1):
            if(z[i,-1]>=0.001):
                k=i
                for j in range(0,Nv+Nc+1):
                    new_const+=z[k][j]*D[j,:]  
                new_const=np.round(new_const,3)
                A=np.vstack([A,new_const[:-1]])
                B=np.hstack((B,new_const[-1]))
                dupli=np.vstack((A.T,B))
                dupli=dupli.T
                dupli2=(np.unique(dupli,axis=0))
                if(np.shape(dupli)[0]==np.shape(dupli2)[0]):
                    break
                else :
                    A=A[:-1,:]
                    B=B[:-1]
        if(i==np.shape(z)[0]-2):
            flag=100
            return A,B,x,sol_tab[-1][-1],flag,new_const
            
        
    return A,B,x,sol_tab[-1][-1],flag,new_const

In [125]:
def w(sol):
    weigh=np.abs(C)
    for a in range(len(C)):
        if math.floor(sol[a])==math.ceil(sol[a]):
            weigh[a]=0
    return weigh

In [126]:
def BAC(A,B,C):
    
    Nv=np.shape(A)[1] # no of variables
    for k in range(1):
        A,B,x,obj,flag,constraint=onecut(A,B,C)
        if flag==1:
            print("Solution complete ",x,obj)
            solution.append(x)
            return x
        elif flag==0: 
            iterations.append(1)
            print("Solution incomplete ",x,obj,"\n Constraint : ",constraint)
        else:
            print("Constraint not found or solution is infeasible ")
    
    
    C0,A0,B0=preprocessing_BAB(C,A,B)
    iterations.append(1)
    sol=solvers.lp(C0,A0,B0)
    
    
    if sol['x'] is None:
        return 0
    sol['x']=np.round(sol['x'],3)
    print(sol['x'])
    
    weight=w(sol['x'])
    i=int(np.argmax(weight))
    d=[math.floor(sol['x'][i]),math.ceil(sol['x'][i])]
    
    if d[0]==d[1]:
        if(np.sum(weight)==0):
            solution.append(sol['x'])
        return sol['x']
    
    
        
    # ceil 
    z=np.zeros(len(C))
    z[i]=-1
    A1=np.vstack([A,z])
    B1=np.append(B,-d[1])
    sol1=BAC(A1,B1,C)
    # checkint(sol1)

    #floor
    z=np.zeros(len(C))
    z[i]=1
    A2=np.vstack([A,z])
    B2=np.append(B,d[0])
    sol2=BAC(A2,B2,C)
    #checkint(sol2)
    
    

In [127]:
def BAB(A,B,C):
    C,A,B=preprocessing_BAB(C,A,B)
    sol=solvers.lp(C,A,B);
    iterations.append(1)
    
    if sol['x'] is None:
        return 0
    sol['x']=np.round(sol['x'],3)
    print(sol['x'])
    
    weight=w(sol['x'])
    i=int(np.argmax(weight))
    d=[math.floor(sol['x'][i]),math.ceil(sol['x'][i])]
    
    if d[0]==d[1]:
        if(np.sum(weight)==0):
            solution.append(sol['x'])
        return sol['x']
    
    
        
    # ceil 
    z=np.zeros(len(C))
    z[i]=-1
    A1=np.vstack([A,z])
    B1=np.append(B,-d[1])
    sol1=BAB(A1,B1,C)
    # checkint(sol1)

    #floor
    z=np.zeros(len(C))
    z[i]=1
    A2=np.vstack([A,z])
    B2=np.append(B,d[0])
    sol2=BAB(A2,B2,C)
    #checkint(sol2)
    
    

Use the following code to run the code,it will give the results and the order of going to nodes which will minimize the
cost also.

In [128]:
C,B,A=preprocessing_TSP(c)
print(C)
if(use==0):
    BAB(A,B,C)   # For branch and bound
else :
    BAC(A,B,C)  # For branch and cut

[ 0. 15. 22.  9. 51. 15.  0.  8. 41.  5. 22.  8.  0.  1. 24.  9. 41.  1.
  0. 57. 51.  5. 24. 57.  0.  0.  0.  0.  0.]
     pcost       dcost       gap    pres   dres   k/t
 0:  1.7537e+02 -6.2223e+02  3e+03  1e+00  1e+00  1e+00
 1: -5.4422e+01 -4.6710e+02  2e+03  5e-01  7e-01  6e+00
 2:  2.8706e+01 -7.3097e+01  3e+02  1e-01  2e-01  4e+00
 3:  4.1040e+01  1.4329e+01  7e+01  3e-02  5e-02  1e+00
 4:  4.6560e+01  3.8144e+01  2e+01  1e-02  1e-02  3e-01
 5:  4.7895e+01  4.6788e+01  3e+00  1e-03  2e-03  4e-02
 6:  4.7985e+01  4.7914e+01  2e-01  9e-05  1e-04  3e-03
 7:  4.8000e+01  4.7999e+01  2e-03  9e-07  1e-06  3e-05
 8:  4.8000e+01  4.8000e+01  2e-05  9e-09  1e-08  3e-07
Optimal solution found.
[[ 0.   ]
 [ 0.25 ]
 [ 0.25 ]
 [ 0.5  ]
 [ 0.   ]
 [ 0.25 ]
 [ 0.   ]
 [ 0.   ]
 [-0.   ]
 [ 0.75 ]
 [ 0.25 ]
 [ 0.   ]
 [ 0.   ]
 [ 0.5  ]
 [ 0.25 ]
 [ 0.5  ]
 [-0.   ]
 [ 0.5  ]
 [-0.   ]
 [-0.   ]
 [ 0.   ]
 [ 0.75 ]
 [ 0.25 ]
 [-0.   ]
 [ 0.   ]
 [ 2.226]
 [ 2.226]
 [ 2.226]
 [ 2.226]]
     pco

 5:  1.0019e+02  9.9798e+01  3e+00  2e-03  2e-03  2e-02
 6:  1.0005e+02  9.9761e+01  2e+00  1e-03  1e-03  1e-02
 7:  1.0000e+02  9.9993e+01  4e-02  2e-05  2e-05  3e-04
 8:  1.0000e+02  1.0000e+02  5e-04  3e-07  3e-07  3e-06
 9:  1.0000e+02  1.0000e+02  5e-06  3e-09  3e-09  3e-08
Optimal solution found.
[[ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [-0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [-0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 4.]
 [ 1.]
 [ 2.]
 [ 3.]]
     pcost       dcost       gap    pres   dres   k/t
 0:  1.4102e+02 -2.9353e+02  4e+03  2e+00  2e+00  1e+00
 1: -1.2773e+01 -1.2953e+02  8e+02  4e-01  5e-01  4e+00
 2:  3.4704e+01 -2.2049e-01  2e+02  1e-01  1e-01  2e+00
 3:  4.8512e+01  3.1539e+01  1e+02  7e-02  7e-02  1e+00
 4:  5.3806e+01  3.9183e+01  1e+02  6e-02  6e-02  1e+00
 5:  6.9976e+01  6.8232e+01  1e+01  7e-03  7e-03  8e-02
 6:  7.0856e+01  7.0668e+01  1e+00  7e-04  7e-04  8e-03
 7:  7.0996e+01  7.0990e+01  5e-02  2e-05  2e-

In [129]:
order=np.zeros(n)
order[0]=1
objective=np.zeros(len(solution))
for j in range(len(solution)):
    x=solution[j]
    objective[j]=np.matmul(C.T,x)
    
optimal=np.argmin(objective)
x=solution[optimal]

#print(x[-n+1:],objective[optimal])
y=x[-n+1:]
for i in range(n-1):
    j=int(y[i])
    order[j]=i+2
print("\nOrder is ",order,"\nObjective is",objective[optimal])


Order is  [1. 4. 3. 5. 2.] 
Objective is 54.0


In [130]:
print("No. of iterations ",len(iterations))
print("Time taken :",time.time()-start_time)


No. of iterations  21
Time taken : 0.6822314262390137
